In [4]:
pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.1.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.1.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd


df = pd.read_csv('alugueis.csv')

df

,id,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
0,0,São Paulo,70,2,1,1,7,acept,furnished,2065,3300,211,42,5618
1,1,São Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750,63,7973
2,2,Porto Alegre,80,1,1,1,6,acept,not furnished,1000,2800,0,41,3841
3,3,Porto Alegre,51,2,1,0,2,acept,not furnished,270,1112,22,17,1421
4,4,São Paulo,25,1,1,0,1,not acept,not furnished,0,800,25,11,836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,10687,Porto Alegre,63,2,1,1,5,not acept,furnished,402,1478,24,22,1926
10607,10688,São Paulo,285,4,4,4,17,acept,not furnished,3100,15000,973,191,19260
10608,10689,Rio de Janeiro,70,3,3,0,8,not acept,furnished,980,6000,332,78,7390
10609,10690,Rio de Janeiro,120,2,2,2,8,acept,furnished,1585,12000,279,155,14020


In [33]:
def amostragem_estratificada(df, amostras, coluna):
    column_values = df[coluna].value_counts().items()
    df_length = len(df)

    percentages = {}
    # I want to loop throught all the column_values and get their name and value do the loop now
    for city, number_of_ocurrences in column_values:
        percentage = number_of_ocurrences / df_length
        percentages[city] = percentage

    samples_df = pd.DataFrame()
    for city, percentage in percentages.items():
        n = int(percentage * amostras)
        df_city = df[df[coluna] == city]
        samples_df = pd.concat([samples_df, df_city.sample(n)])
        
    return samples_df

amostragem_estratificada(df, 10, 'city')

,id,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
9936,10008,São Paulo,45,1,1,0,1,not acept,not furnished,0,1100,0,17,1117
8587,8653,São Paulo,105,2,1,0,-,acept,not furnished,0,1800,125,28,1953
1263,1280,São Paulo,427,4,6,4,13,acept,not furnished,3440,10000,1350,127,14920
10193,10269,São Paulo,300,4,3,1,9,not acept,furnished,2300,8000,768,102,11170
5855,5901,São Paulo,376,5,6,4,13,acept,not furnished,4252,12000,1980,153,18390
6683,6739,Rio de Janeiro,30,1,1,0,10,acept,furnished,340,1386,34,18,1778
8269,8334,Belo Horizonte,600,3,5,8,-,acept,not furnished,800,10000,202,164,11170
7594,7654,Porto Alegre,70,3,2,1,7,acept,not furnished,530,2300,111,34,2975
